In [1]:
import numpy as np
import pylab as pl
import torch
import torch.optim as optim
import torch.nn as nn
import random
from collections import OrderedDict

torch.set_printoptions(edgeitems=2)

In [2]:
#参数
num=100#畴数
b_max=4.0e12#最大带宽
l=3e-6#畴长

In [3]:
# 定义函数

#折射率
def n(omega):
    if(omega>1e14):
        t=100
        a1=4.582
        a2=0.09921
        a3=0.2109
        a4=-0.02194
        b1=0.00000022971
        b2=0.000000052716
        b3=-0.000000049143
        c=88506.25
        t1=t+273.15
        
        w=(2*np.pi*3e8/omega)*1e6
        c1=w*w-(a3+b3*(t1*t1-c))*(a3+b3*(t1*t1-c))
        c2=a1+b1*(t1*t1-c)+(a2+b2*(t1*t1-c))/c1+a4*w*w
        y=np.sqrt(c2)
    else:
        d1=19.9
        d2=44
        d3=4.533e12*2*np.pi
        d4=0.426e12*2*np.pi
        d5=2.9176e13
        
        e=d1+(d2-d1)*d5**2/(d3**2+1.0j*omega*d4-omega**2)
        y=np.real(np.sqrt(e))

    return y

#波矢
def kk(f):
    z=f*n(f)/3e8
    return z

#带宽计算
def funct(val):
    G=np.zeros((2,500),dtype=np.complex)
    d=np.zeros((2,num+1),dtype=np.complex)
        
    d[1,:-1] = val
    d[1,num] = 0
    
        
    for j in range(num+1):
        d[0,j]=j*l
        
    for k in range(500):
        f_thz=0.1e12+k*(b_max-0.1e12)/499
        f_nir=3e8/800e-9
        G[0,k]=-kk(2*np.pi*f_nir)+kk(2*np.pi*(f_nir-f_thz))+kk(2*np.pi*f_thz)
        
    for ii in np.arange(1,500):
        for jj in np.arange(1,num):
            G[1,ii]=G[1,ii]+d[1,jj]*(np.exp(1.0j*G[0,ii]*d[0,jj+1])-np.exp(1.0j*G[0,ii]*d[0,jj]))/(1.0j*G[0,ii])
        
    peak = np.max(np.abs(G[1,:]))
    G_au = np.abs(G[1,:])/np.max(np.abs(G[1,:]))
        
    max_index = G_au.tolist().index(1)
        
    lo = max_index 
    # 前向搜索
    while G_au[lo] > 0.5 and lo > 0:
        lo -= 1
        if G_au[lo] <= 0.5:
            break 

    hi = max_index
    # 后向搜索
    while G_au[hi] > 0.5 and hi < 499:
        hi += 1
        if G_au[hi] <= 0.5:
            break

    if(hi==499 and G_au[hi]>0.5)or(lo==0 and G_au[lo]>0.5):
        bw=-1
    else:
        bw = hi - lo
    
    bandwidth=bw*(b_max-0.1e12)/499e12
    return bandwidth

In [4]:
# 定义拟合网络：将输入的变量映射到最后的值：
class FitModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
                nn.Linear(num, 300),
                nn.ReLU(),
                nn.Linear(300, 500),
                nn.ReLU(),
                nn.Linear(500, 300),
                nn.ReLU(),
                nn.Linear(300, 1)
            )
        
        
    def forward(self, x):
        x = self.net(x)
        return x

# 将定义的模型实例化：
fitNet = FitModel()
# 为实例化后的模型定义优化器
fitOpt = optim.SGD(fitNet.parameters(), lr=0.01)

In [5]:
fitNet

FitModel(
  (net): Sequential(
    (0): Linear(in_features=100, out_features=300, bias=True)
    (1): ReLU()
    (2): Linear(in_features=300, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=300, bias=True)
    (5): ReLU()
    (6): Linear(in_features=300, out_features=1, bias=True)
  )
)

In [37]:
# 定义生成网络：生成100个输入值：

# 由于输入值只能为1或-1，所以选取tanh函数，通过对x放大，可以将实数域的值
# 映射到近似1或-1两个值（可以绘制tanh(100x)的图形帮助理解）
# 于是定义如下激活函数：
class MyTanh(nn.Module):
    def __init__(self):
        super(MyTanh, self).__init__()
        
    def forward(self, x):
        x = torch.tanh(num*x)
        return x 

# 接下来定义生成网络：
class GenModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 网络1：得出输入        
        self.net = nn.Sequential(
                nn.Linear(num, 300),
                nn.LeakyReLU(),
                nn.Linear(300, 500),
                nn.LeakyReLU(),
                nn.Linear(500, 600),
                nn.LeakyReLU(),
                nn.Linear(600, 500),
                nn.LeakyReLU(),
                nn.Linear(500, 300),
                nn.LeakyReLU(),
                nn.Linear(300, 100),
                MyTanh()
            )
        
    def forward(self, x):
        x = self.net(x)
        return x

genNet = GenModel()
genOpt = optim.SGD(genNet.parameters(), lr=0.05)

In [7]:
# 输出获取函数
def getOutput(inList):
    G=np.zeros([2,500],dtype=complex)
    d=np.zeros([2,num+1],dtype=complex)

    d[1,0:-1] = np.array(inList)
    d[1,num]=0

    for j in range(num+1):
        d[0,j]=j*l

    for k in range(500):
        f_thz=0.1e12+k*(b_max-0.1e12)/499
        f_nir=3e8/800e-9
        G[0,k]=-kk(2*np.pi*f_nir)+kk(2*np.pi*(f_nir-f_thz))+kk(2*np.pi*f_thz)

    for ii in np.arange(1,500):
        for jj in np.arange(1,num):
            G[1,ii]=G[1,ii]+d[1,jj]*(np.exp(1.0j*G[0,ii]*d[0,jj+1])-np.exp(1.0j*G[0,ii]*d[0,jj]))/(1.0j*G[0,ii])

    peak = np.max(np.abs(G[1,:]))  
    G_au = np.abs(G[1,:])/np.max(np.abs(G[1,:]))
    max_index = G_au.tolist().index(1.0)

    lo = max_index 
    # 前向搜索
    while G_au[lo] >= 0.5 and lo > 0:
        lo -= 1
        if G_au[lo] < 0.5:
            break 

    hi = max_index
    # 后向搜索
    while G_au[hi] >= 0.5 and hi < 499:
        hi += 1
        if G_au[hi] < 0.5:
            break

    if(hi==499 and G_au[hi]>0.5):
        bw=500-lo
    elif(lo==0 and G_au[lo]>0.5):
        bw=hi-0
    else:
        bw = hi-lo        
    
    bandwidth=bw*(b_max-0.1e12)/499e12
    return bandwidth

# 输入输出获取函数
def generateInput(size):
    inputList = []
    outputList=[]
    i=0
    while i<size: 
        inList = []
        for j in range(num):
            item = random.randint(0, 1)
            if item == 0:
                item = -1
            inList.append(item)
        outList=getOutput(inList)
        i=i+1
        inputList.append(inList)
        outputList.append(outList)
        
    return inputList,outputList

In [8]:
# 获取输入数据与输出数据：
inputList,outputList = generateInput(1000)

In [26]:
outputList

tensor([[0.1407],
        [0.0782],
        [0.5002],
        [0.1251],
        [0.2579],
        [0.2267],
        [0.1798],
        [0.0625],
        [0.1954],
        [0.3126],
        [0.2579],
        [0.1094],
        [0.1798],
        [0.1251],
        [0.0547],
        [0.1954],
        [0.0625],
        [0.1876],
        [0.2579],
        [0.2735],
        [0.0547],
        [0.1172],
        [0.0782],
        [0.1172],
        [0.1798],
        [0.1719],
        [0.0860],
        [0.1719],
        [0.1329],
        [0.2657],
        [0.1798],
        [0.0782],
        [0.2345],
        [0.3517],
        [0.2657],
        [0.1719],
        [0.1563],
        [0.1329],
        [0.2188],
        [0.1954],
        [0.5549],
        [0.2501],
        [0.1329],
        [0.2032],
        [0.3439],
        [0.1876],
        [0.0938],
        [0.1094],
        [0.1563],
        [0.1407],
        [0.1563],
        [0.2501],
        [0.2423],
        [0.0703],
        [0.3126],
        [0

In [27]:
inList=[-1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,
         1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,
        -1., -1., -1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1.,  1.,
        -1., -1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,
         1.,  1., -1., -1., -1.,  1., -1., -1., -1., -1.,  1.,  1.,  1., -1.,
         1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,  1., -1.,
         1., -1., -1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,
        -1., -1.]

getOutput(inList)

0.0937875751503006

In [28]:
# 对输入数据与输出数据tensor包装，以便输入神经网络进行训练：
inputList = torch.tensor(inputList)
outputList = torch.tensor(outputList).unsqueeze(1)

# 进行训练集与测试集划分：
n_samples = inputList.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

input_train = inputList[train_indices]     # 训练集：输入数据
output_train = outputList[train_indices]   # 训练集：输出数据

input_val = inputList[val_indices]         # 测试集：输入数据
output_val = outputList[val_indices]       # 测试集：输出数据

C:\Users\20241\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
C:\Users\20241\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
# 定义拟合网络的训练函数：
def fitTrain(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 50 == 0:
            print('Epoch {}, Training loss {}, Validation loss {}'.format(
                epoch, float(loss_train), float(loss_val)))

In [30]:
fitNet

FitModel(
  (net): Sequential(
    (0): Linear(in_features=100, out_features=300, bias=True)
    (1): ReLU()
    (2): Linear(in_features=300, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=300, bias=True)
    (5): ReLU()
    (6): Linear(in_features=300, out_features=1, bias=True)
  )
)

In [31]:
# 进行拟合函数的训练：
fitTrain(
    n_epochs = 200, 
    optimizer = fitOpt,
    model = fitNet,
    loss_fn = nn.MSELoss(),
    t_u_train = input_train.float(),
    t_u_val = input_val.float(), 
    t_c_train = output_train.float(),
    t_c_val = output_val.float())

C:\Users\20241\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([800, 1, 1])) that is different to the input size (torch.Size([800, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\20241\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([200, 1, 1])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Training loss 0.014616081491112709, Validation loss 0.018139345571398735
Epoch 50, Training loss 0.01458809245377779, Validation loss 0.018125319853425026
Epoch 100, Training loss 0.01456172950565815, Validation loss 0.018105003982782364
Epoch 150, Training loss 0.014537307433784008, Validation loss 0.01808582805097103
Epoch 200, Training loss 0.014514596201479435, Validation loss 0.018067918717861176


In [32]:
# 拟合网络训练结束后，冻结其全部参数，以免在生成网络的训练过程中被修改：
fitNet.require_grad = False

In [33]:
# 定义生成网络的训练：
def genTrain(n_epochs, optimizer, model):
    val = torch.ones(num)
    out = 0
    for epoch in range(1, n_epochs+1):
        out = model(val)
        
        # 我们希望fitNet(out)的值越大越好
#         loss = torch.exp(1/fitNet(out))
        loss = -fitNet(out)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print('Epoch {}, loss {}, out {}'.format(
                epoch, float(loss), out))
    return out

In [38]:
# 进行生成网络的训练：
out = genTrain(5000, genOpt, genNet)

Epoch 1, loss -0.15582701563835144, out tensor([-0.7077, -0.9999,  0.6496, -1.0000, -0.8532, -0.9996,  0.9142, -1.0000,
         0.9988,  0.9985, -1.0000,  0.9996,  0.9970,  0.9989, -1.0000,  0.9887,
         1.0000, -0.9393, -0.7495,  0.9829,  0.9999,  0.9833, -0.9961, -0.8786,
        -0.6983,  1.0000,  1.0000,  0.9909, -0.5559,  0.2996,  0.8084,  0.9919,
         0.9836,  0.9964,  0.9998,  1.0000, -0.9961,  1.0000,  0.9937,  0.7204,
         0.8879, -0.9994, -0.7333,  0.9931, -0.3534, -0.8850,  0.9999,  0.9188,
         1.0000,  0.8583,  0.9757, -0.7806, -0.4498, -0.9998,  0.9247, -0.9997,
         0.7112, -0.0168, -1.0000, -0.9871, -0.9999, -0.9729,  0.9973, -0.8948,
         0.9705, -0.9934, -0.3702,  1.0000,  0.5612,  0.9717, -0.8672, -0.9996,
        -0.8964,  0.9028, -0.9969, -1.0000,  0.9960,  0.5293,  0.5034,  0.9950,
        -0.9997,  1.0000,  0.9999,  0.8454, -1.0000,  0.9988, -0.9998,  0.8877,
         0.7981,  0.9876,  0.9995,  0.6453, -0.4218,  1.0000, -0.8261,  0.9989,


In [39]:
out

tensor([-1.0000, -0.9610,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         1.0000, -0.9999, -1.0000, -0.9999,  1.0000,  0.9999, -1.0000, -0.9998,
         1.0000, -1.0000,  0.9999,  1.0000, -0.8768, -0.9998, -1.0000,  0.4814,
        -0.9999, -0.9995,  1.0000, -0.9998,  1.0000,  0.9999, -0.9999, -0.8439,
        -0.9999,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  0.9216,  1.0000,
         1.0000, -0.6207,  1.0000,  0.9998, -1.0000,  0.9999,  0.9999, -0.9990,
         1.0000, -0.9993, -0.9604,  1.0000,  0.9999,  0.9999, -1.0000,  1.0000,
        -1.0000,  1.0000, -1.0000, -0.9999, -1.0000,  0.9999, -0.9999, -0.9999,
        -0.9999,  1.0000, -1.0000,  1.0000,  1.0000, -0.9830,  0.9999, -0.9172,
         0.9997, -0.9999,  1.0000, -1.0000,  0.9999,  0.9999, -1.0000, -1.0000,
         0.9998,  1.0000,  1.0000,  0.9999, -1.0000,  0.9999, -1.0000,  1.0000,
        -0.9999,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000, -1.0000, -0.9999,
        -1.0000, -1.0000,  1.0000, -1.00